# watsonx.aiのLLMでLangChainとMilvusを使ってPDFの内容をQ&Aしてみた(=RAG)

https://qiita.com/nishikyon/items/8fa659d6392357925a92/

# 必要なライブラリのインストール

In [ ]:
# 既にインストール済みであればはこのコードの実行は不要です

!pip install 'ibm-watsonx-ai>=1.1.15'
!pip install 'langchain>=0.3.3'
!pip install 'langchain-ibm>=0.3.1'
!pip install 'langchain-huggingface>=0.1.0'
!pip install 'langchain-milvus>=0.1.6'
!pip install 'langchain-community>=0.3.2'
!pip install 'pypdf>=5.0.0'
!pip install 'pymilvus>=2.4.8'

**インストール終了後、一旦カーネルを再起動してください**

#  (オプション)LangChainで使えるLLMの確認


In [ ]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

# LangChainで使えるLLMの取得

In [2]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import WatsonxLLM

watsonx_url = "https://jp-tok.ml.cloud.ibm.com" # watsonx.aiのAuthentication用のエンドポイントのURL
apikey = "<APIキー>"
project_id = "<PROJECT ID>"

# 使用するLLMのパラメータ
generate_params = {
    GenParams.MAX_NEW_TOKENS: 500,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1
}

# LangChainで使うllm
custom_llm = WatsonxLLM(
    model_id="ibm/granite-8b-japanese", #使用するLLM名
    url=watsonx_url,
    apikey=apikey,
    project_id=project_id,
    params=generate_params,
)

# そのまま実行

In [ ]:
result=custom_llm.invoke("IBM Db2 on Cloudの特徴は?")
print(result)

# PDFの取得

In [ ]:
# Milvusに保存した内容を使う場合はこのコードの実行は不要です

!wget https://files.speakerdeck.com/presentations/cc34f85fe9b5467d8782a41e5fa39b78/Dojo_Db2RESTAPI_20230727_%E9%85%8D%E5%B8%83%E7%94%A8.pdf

# PDFLoaderの作成

In [4]:
# Milvusに保存した内容を使う場合はこのコードの実行は不要です

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./Dojo_Db2RESTAPI_20230727_配布用.pdf") #ダウンロードしたPDFを指定

#  PDF ドキュメントの内容をページで分割する

In [5]:
# Milvusに保存した内容を使う場合はこのコードの実行は不要です

pages = loader.load_and_split() 

#  embeddingsの取得

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

# MilvusにデータをInsert

## Milvusへの接続情報を`connection_args`に辞書形式でセットします

In [ ]:
my_connection_args ={
 'uri':'http://localhost:19530', 
}

In [8]:
my_connection_args ={
 'uri':'https://xxxxxx.ibm.com:35382', 
 'token':'yyyyyyyy:zzzzzzzz',
 # 'server_pem_path':'/Users/nishito/presto.crt',  # watsonx.data SaaS Milvusは不要
 # 'server_name':'watsonxdata'　# watsonx.data SaaS Milvusは不要
}

## Documentをベクトル化して保存 (Milvus.from_documents)

In [10]:
# Milvusに保存した内容を使う場合はこのコードの実行は不要です
from langchain_milvus import Milvus

vector_db = Milvus.from_documents(
    pages,
    embeddings,
    connection_args=my_connection_args,
    drop_old=True, #追加の場合はここをFalseに
    collection_name = 'LangChainCollection'
)

### Milvusへの接続(DBにInsert不要の場合)

In [11]:
from langchain_milvus import Milvus

vector_db = Milvus(
    embeddings,
    connection_args=my_connection_args,
    collection_name = 'LangChainCollection'
)

# テキスト類似検索してみます

In [ ]:
query = "IBM TechXchange Japanとは?"
docs = vector_db.similarity_search(query)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )

In [ ]:
# 取得数をkで指定
docs = vector_db.similarity_search(query, k=1)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )

In [ ]:
# ススコアは小さいほど(0に近いほど)類似度が高いです。
query = "IBM TechXchange Japanとは?"
docs = vector_db.similarity_search_with_score(query)
for doc, score in docs:
    print({"score": score, "content": doc.page_content[0:100], "metadata": doc.metadata} )

# watsonx.aiのLLMでLangChainとMilvusを使ってPDFの内容をQ&A

## まずはプロンプトなしでQ&A

In [ ]:
from langchain.chains import RetrievalQA

retriever = vector_db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=custom_llm, chain_type="stuff", retriever=retriever)
query = "IBM TechXchange Japanとは?" 
answer = qa.invoke(query)
print(answer['result'])

In [ ]:
query = "日本の首都は?" 
answer = qa.invoke(query)
print(answer['result'])

## プロンプトを作ってQ&A

### RAG chainの作成　

In [17]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

template = """以下のcontextのみを利用して、ですます調で丁寧に回答してください。contextと質問が関連していない場合は、「不明です。」と回答お願いします。
context: {context}
質問: {question}
回答:"""

rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | custom_llm
)

### 質問してみます

In [ ]:
print(rag_chain.invoke("日本の首都は？"))

In [ ]:
print(rag_chain.invoke("IBM TechXchange Japanとは？"))

### プロンプトちょっと変えてみます

In [21]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

template2 = """<|system|>
あなたはIBMが開発したAI言語モデル、Granite Chatです。あなたは慎重なアシスタントです。あなたは慎重に指示に従います。あなたは親切で無害で、倫理的なガイドラインに従い、前向きな行動ができます。
<|user|>
あなたは、特別なRetrieval Augmented Generation（RAG）アシスタントとして機能するように設計されたAI言語モデルです。応答を生成するときは、正しさを優先します。つまり、文脈とユーザーのクエリが与えられたときに応答が正しく、文脈に根拠があることを確認します。さらに、レスポンスが与えられたドキュメントまたはコンテキストによってサポートされていることを確認してください。コンテキストやドキュメントを使用して質問に答えることができない場合、次のレスポンスを出力します: 'わかりません。' あなたの回答が質問に関連していることを常に確認してください。説明が必要な場合は、まず説明や理由を述べ、それから最終的な答えを述べてください。
[文書]
{context}
[質問]
{question}
<|assistant|>"""

rag_prompt = PromptTemplate.from_template(template2)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | custom_llm
)

In [ ]:
print(rag_chain.invoke("IBM TechXchange Japanとは？"))

In [ ]:
print(rag_chain.invoke("日本の首都は？"))